In [16]:
#@title Set up working directories and download files/checkpoints. { display-mode: "form" }
# Create directory for storing model weights (2.3GB) and example sequences.
# Here we use the encoder-part of ProtT5-XL-U50 in half-precision (fp16) as 
# it performed best in our benchmarks (also outperforming ProtBERT-BFD).
# Also download secondary structure prediction checkpoint to show annotation extraction from embeddings
!mkdir protT5 # root directory for storing checkpoints, results etc
!mkdir protT5/protT5_checkpoint # directory holding the ProtT5 checkpoint
!mkdir protT5/subcell_checkpoint # directory storing the supervised classifier's checkpoint
!mkdir protT5/output # directory for storing your embeddings & predictions
!wget -nc -P protT5/ https://rostlab.org/~deepppi/example_seqs.fasta
!wget -nc -P protT5/protT5_checkpoint https://rostlab.org/~deepppi/protT5_xl_u50_encOnly_fp16_checkpoint/pytorch_model.bin
!wget -nc -P protT5/protT5_checkpoint https://rostlab.org/~deepppi/protT5_xl_u50_encOnly_fp16_checkpoint/config.json
# Huge kudos to the bio_embeddings team here! We will integrate the new encoder, half-prec ProtT5 checkpoint soon
!wget -nc -P protT5/subcell_checkpoint http://data.bioembeddings.com/public/embeddings/feature_models/t5/subcell_checkpoint.pt

--2023-12-09 10:15:20--  https://rostlab.org/~deepppi/example_seqs.fasta
Resolving rostlab.org (rostlab.org)... 131.159.28.73
Connecting to rostlab.org (rostlab.org)|131.159.28.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 627
Saving to: ‘protT5/example_seqs.fasta’

example_seqs.fasta  100%[===================>]     627  --.-KB/s    in 0s      

2023-12-09 10:15:21 (199 MB/s) - ‘protT5/example_seqs.fasta’ saved [627/627]

--2023-12-09 10:15:21--  https://rostlab.org/~deepppi/protT5_xl_u50_encOnly_fp16_checkpoint/pytorch_model.bin
Resolving rostlab.org (rostlab.org)... 131.159.28.73
Connecting to rostlab.org (rostlab.org)|131.159.28.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2416373051 (2,2G) [application/octet-stream]
Saving to: ‘protT5/protT5_checkpoint/pytorch_model.bin’

pytorch_model.bin    54%[=========>          ]   1,23G  11,9MB/s    eta 89s    ^C
--2023-12-09 10:17:08--  https://rostlab.org/~deepppi/protT5_xl_u50_

In [5]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";
seq_path = "./protT5/example_seqs.fasta"

# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = False 
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = False
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
subcell_mem = True
subcell_path = "./protT5/output/subcell.csv" # file for storing predictions
mem_path = "./protT5/output/membrane.csv" # file for storing predictions
# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or subcell_mem is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")


mkdir: protT5: File exists
mkdir: protT5/protT5_checkpoint: File exists
mkdir: protT5/sec_struct_checkpoint: File exists
mkdir: protT5/output: File exists
File ‘protT5/example_seqs.fasta’ already there; not retrieving.

File ‘protT5/sec_struct_checkpoint/secstruct_checkpoint.pt’ already there; not retrieving.



In [4]:
from transformers import T5EncoderModel, T5Tokenizer
import torch
from torch import nn
import h5py
import time
import csv

/Users/ugur_dura/opt/anaconda3/envs/protein_prediction/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# In the following you can define your desired output. Current options:
# per_residue embeddings
# per_protein embeddings
# secondary structure predictions

# Replace this file with your own (multi-)FASTA
# Headers are expected to start with ">";
seq_path = "./protT5/example_seqs.fasta"

# whether to retrieve embeddings for each residue in a protein 
# --> Lx1024 matrix per protein with L being the protein's length
# as a rule of thumb: 1k proteins require around 1GB RAM/disk
per_residue = True 
per_residue_path = "./protT5/output/per_residue_embeddings.h5" # where to store the embeddings

# whether to retrieve per-protein embeddings 
# --> only one 1024-d vector per protein, irrespective of its length
per_protein = True
per_protein_path = "./protT5/output/per_protein_embeddings.h5" # where to store the embeddings

# whether to retrieve secondary structure predictions
# This can be replaced by your method after being trained on ProtT5 embeddings
sec_struct = True
sec_struct_path = "./protT5/output/ss3_preds.fasta" # file for storing predictions

# make sure that either per-residue or per-protein embeddings are stored
assert per_protein is True or per_residue is True or sec_struct is True, print(
    "Minimally, you need to active per_residue, per_protein or sec_struct. (or any combination)")

In [7]:

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print("Using {}".format(device))

Using mps


In [8]:
#@title Network architecture for secondary structure prediction. { display-mode: "form" }
# Convolutional neural network (two convolutional layers) to predict secondary structure
class ConvNet( torch.nn.Module ):
    def __init__( self ):
        super(ConvNet, self).__init__()
        # This is only called "elmo_feature_extractor" for historic reason
        # CNN weights are trained on ProtT5 embeddings
        self.elmo_feature_extractor = torch.nn.Sequential(
                        torch.nn.Conv2d( 1024, 32, kernel_size=(7,1), padding=(3,0) ), # 7x32
                        torch.nn.ReLU(),
                        torch.nn.Dropout( 0.25 ),
                        )
        n_final_in = 32
        self.dssp3_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 3, kernel_size=(7,1), padding=(3,0)) # 7
                        )
        
        self.dssp8_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 8, kernel_size=(7,1), padding=(3,0))
                        )
        self.diso_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 2, kernel_size=(7,1), padding=(3,0))
                        )
        

    def forward( self, x):
        # IN: X = (B x L x F); OUT: (B x F x L, 1)
        x = x.permute(0,2,1).unsqueeze(dim=-1) 
        x         = self.elmo_feature_extractor(x) # OUT: (B x 32 x L x 1)
        d3_Yhat   = self.dssp3_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 3)
        d8_Yhat   = self.dssp8_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 8)
        diso_Yhat = self.diso_classifier(  x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 2)
        return d3_Yhat, d8_Yhat, diso_Yhat

In [9]:
def load_subcell_model():
  checkpoint_dir="./protT5/subcell_checkpoint/subcell_checkpoint.pt"
  state = torch.load( checkpoint_dir )
  model = FNN()
  model.load_state_dict(state['state_dict'])
  model = model.eval()
  model = model.half()
  model = model.to(device)
  return model

In [10]:
def get_T5_model():
    model = T5EncoderModel.from_pretrained("./protT5/protT5_checkpoint/", torch_dtype=torch.float16)
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False ) 

    return model, tokenizer

In [11]:
csv_file_path = '../merged.csv'  

seqs = {}    

with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        uniprot_id = row['Uniprot_ID']
        protein_sequence = row['Protein_sequence']
        
        seqs[uniprot_id] = protein_sequence

In [12]:
def get_embeddings( model, tokenizer, seqs, per_residue, per_protein, subcell_mem, 
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    if subcell_mem:
      subcell_model = load_subcell_model()

    results = {"residue_embs" : dict(), 
               "protein_embs" : dict(),
               "subcell" : dict(),
               "mem" : dict(), 
               }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
                    # returns: ( batch-size x max_seq_len_in_minibatch x embedding_dim )
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue

            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                # slice off padding --> batch-size x seq_len x embedding_dim  
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]

                if subcell_mem: # in case you want to predict secondary structure from embeddings
                  subcell_Yhat, mem_Yhat = subcell_model(emb.mean(dim=0,keepdims=True))
                  results["subcell"][identifier] = torch.max( subcell_Yhat, dim=1)[1].detach().cpu().numpy().squeeze()
                  results["mem"][identifier] = torch.max( mem_Yhat, dim=1)[1].detach().cpu().numpy().squeeze()


                if per_residue: # store per-residue embeddings (Lx1024)
                    results["residue_embs"][ identifier ] = emb.detach().cpu().numpy().squeeze()
                if per_protein: # apply average-pooling to derive per-protein embeddings (1024-d)
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()


    passed_time=time.time()-start
    avg_time = passed_time/len(seq_dict)
    print('\n############# EMBEDDING STATS #############')
    print('Total number of per-residue embeddings: {}'.format(len(results["residue_embs"])))
    print('Total number of per-protein embeddings: {}'.format(len(results["protein_embs"])))
    print("Time for generating embeddings: {:.1f}[m] ({:.3f}[s/protein])".format(
        passed_time/60, avg_time ))
    print('\n############# END #############')
    return results

In [13]:
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [14]:
def write_prediction_csv(predictions, out_path, mode):
  # Label mapping for subcellular localization
  subcell_mapping = {
      0: "Cell_Membrane",
      1: "Cytoplasm",
      2: "Endoplasmatic Reticulum",
      3: "Golgi Apparatus",
      4: "Lysosome or vacuole",
      5: "Mitochondrion",
      6: "Nucleus",
      7: "PEROXISOME",
      8: "Plastid",
      9: "Extracellular"
  }
  # Label mapping for membrane-bound
  mem_mapping = {
      0: "Soluble",
      1: "Membrane-bound"
  }

  if mode=="subcell":
    class_mapping=subcell_mapping
  elif mode=="mem":
    class_mapping=mem_mapping
  else:
    raise NotImplemented

  with open(out_path, 'w+') as out_f:
      out_f.write( '\n'.join( 
          [ "{},{}".format( 
              seq_id, class_mapping[int(yhat)]) 
          for seq_id, yhat in predictions.items()
          ] 
            ) )
  return None

In [15]:
model, tokenizer = get_T5_model()

OSError: ./protT5/protT5_checkpoint/ does not appear to have a file named config.json. Checkout 'https://huggingface.co/./protT5/protT5_checkpoint//main' for available files.